In [1]:
#i3d extraction

!git clone https://github.com/piergiaj/pytorch-i3d.git
%cd pytorch-i3d
!pip install torch torchvision numpy pillow tqdm

# Create models directory
!mkdir -p models

# Download flow_imagenet.pt into models/
!wget -O /kaggle/working/flow_imagenet.pt https://www.dropbox.com/s/7w4z5q9fowcp9x5/flow_imagenet.pt?dl=1


Cloning into 'pytorch-i3d'...
remote: Enumerating objects: 21, done.
remote: Total 21 (delta 0), reused 0 (delta 0), pack-reused 21 (from 1)
Receiving objects: 100% (21/21), 178.18 MiB | 41.07 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Updating files: 100% (12/12), done.
/kaggle/working/pytorch-i3d
--2025-04-08 17:45:06--  https://www.dropbox.com/s/7w4z5q9fowcp9x5/flow_imagenet.pt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/kaggle/working/flow_imagenet.pt’

/kaggle/working/flo     [ <=>                ]  70.62K  --.-KB/s    in 0.06s   

2025-04-08 17:45:06 (1.08 MB/s) - ‘/kaggle/working/flow_imagenet.pt’ saved [72310]



In [2]:
!ls /kaggle/working/pytorch-i3d

charades_dataset_full.py  LICENSE.txt	  README.md
charades_dataset.py	  models	  train_i3d.py
extract_features.py	  pytorch_i3d.py  videotransforms.py


In [3]:
import sys
sys.path.append('/kaggle/working/pytorch-i3d')
from pytorch_i3d import InceptionI3d



!wget -O /kaggle/working/flow_imagenet.pt https://github.com/piergiaj/pytorch-i3d/raw/master/models/flow_imagenet.pt
!file /kaggle/working/flow_imagenet.pt
!wget -O /kaggle/working/rgb_imagenet.pt https://github.com/piergiaj/pytorch-i3d/raw/master/models/rgb_imagenet.pt


--2025-04-08 17:45:09--  https://github.com/piergiaj/pytorch-i3d/raw/master/models/flow_imagenet.pt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/piergiaj/pytorch-i3d/master/models/flow_imagenet.pt [following]
--2025-04-08 17:45:10--  https://raw.githubusercontent.com/piergiaj/pytorch-i3d/master/models/flow_imagenet.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50795330 (48M) [application/octet-stream]
Saving to: ‘/kaggle/working/flow_imagenet.pt’

/kaggle/working/flo 100%[===================>]  48.44M   142MB/s    in 0.3s    

2025-04-08 17:45:10 (142 MB/s) - ‘/kaggle/working/flow_imagen

# FRAME FEATURE EXTRACTION

In [4]:
import os
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as transforms
from pytorch_i3d import InceptionI3d

# --- Config ---
dataset_path = '/kaggle/input/shanghaitech-anomaly-detection/dataset/mp'
output_dir = '/kaggle/working/I3D_Features_rgbframetrain'
os.makedirs(output_dir, exist_ok=True)

# --- Device Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- I3D Model Loader ---
def load_i3d_model():
    # Load RGB I3D with ImageNet pretrained weights
    i3d = InceptionI3d(400, in_channels=3)
    i3d.load_state_dict(torch.load('/kaggle/working/rgb_imagenet.pt')) 
    i3d.to(device)
    i3d.eval()
    return i3d

# --- I3D Feature Extractor (1024-D) ---
class I3DFeatureExtractor(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        # Use the actual layer names from the InceptionI3d model
        self.features = torch.nn.Sequential(
            model.Conv3d_1a_7x7,
            model.MaxPool3d_2a_3x3,
            model.Conv3d_2b_1x1,
            model.Conv3d_2c_3x3,
            model.MaxPool3d_3a_3x3,
            model.Mixed_3b,
            model.Mixed_3c,
            model.MaxPool3d_4a_3x3,
            model.Mixed_4b,
            model.Mixed_4c,
            model.Mixed_4d,
            model.Mixed_4e,
            model.Mixed_4f,
            model.MaxPool3d_5a_2x2,
            model.Mixed_5b,
            model.Mixed_5c
        )

    def forward(self, x):
        x = x.unsqueeze(2)  # (N, C, 1, H, W)
        x = self.features(x)
        x = torch.mean(x, dim=[2, 3, 4])  # Global average pooling
        return x

# --- Image Preprocessing ---
def preprocess_frame(frame_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # I3D expects 224x224
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # [-1,1]
    ])
    img = Image.open(frame_path).convert('RGB')
    return transform(img).to(device)

# --- Feature Extraction ---
def extract_i3d_features(model, dataset_path, output_dir):
    extractor = I3DFeatureExtractor(model)
    
    for video_folder in tqdm(sorted(os.listdir(dataset_path))):
        frame_dir = os.path.join(dataset_path, video_folder, 'frames')
        if not os.path.exists(frame_dir):
            continue
            
        frame_paths = sorted([os.path.join(frame_dir, f) 
                           for f in os.listdir(frame_dir) 
                           if f.lower().endswith(('.jpg','.png','.jpeg'))])
        
        # Create output directory for this video's features
        video_output_dir = os.path.join(output_dir, video_folder)
        os.makedirs(video_output_dir, exist_ok=True)
        
        for frame_idx, frame_path in enumerate(frame_paths):
            try:
                frame = preprocess_frame(frame_path)
                with torch.no_grad():
                    features = extractor(frame.unsqueeze(0)).cpu().numpy()  # (1,1024)
        
                
                # Save each frame's features as separate numpy file
                frame_features = features.squeeze()
                np.save(os.path.join(video_output_dir, f"frame_{frame_idx:04d}_i3d.npy"), frame_features)
                
            except Exception as e:
                print(f"Error processing {frame_path}: {e}")
                continue

if __name__ == "__main__":
    print("Loading I3D model...")
    i3d = load_i3d_model()
    
    print("Extracting 1024-D features...")
    extract_i3d_features(i3d, dataset_path, output_dir)
    
    # Verify feature dimensions
    sample_video = os.listdir(output_dir)[0]
    sample_frame = os.listdir(os.path.join(output_dir, sample_video))[0]
    sample_features = np.load(os.path.join(output_dir, sample_video, sample_frame))
    print(f"\nFeature verification:")
    print(f"Shape: {sample_features.shape}")  # Should be (1024,)
    print(f"First 5 dimensions: {sample_features[:5]}")
    
    print(f"\n✅ 1024-D features saved to individual files in {output_dir}")

Using device: cuda
Loading I3D model...


<ipython-input-4-e506caca072f>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  i3d.load_state_dict(torch.load('/kaggle/working/rgb_imagenet.pt'))


Extracting 1024-D features...


100%|██████████| 330/330 [1:58:41<00:00, 21.58s/it]


Feature verification:
Shape: (1024,)
First 5 dimensions: [0.01181234 0.0151756  0.15537691 0.07811625 1.082841  ]

✅ 1024-D features saved to individual files in /kaggle/working/I3D_Features_rgbframetrain


# FLOW FEATURE EXTRACTION

In [5]:
import os
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as transforms
from pytorch_i3d import InceptionI3d

# --- Config ---
dataset_path = '/kaggle/input/shanghaitech-anomaly-detection/dataset/mp'
output_dir = '/kaggle/working/I3D_Features_flowframetrain'
os.makedirs(output_dir, exist_ok=True)

# --- Device Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- I3D Model Loader ---
def load_i3d_model():
    # Load Flow I3D with ImageNet pretrained weights
    # Note: Flow model uses 2 channels instead of 3
    i3d = InceptionI3d(400, in_channels=2)
    i3d.load_state_dict(torch.load('/kaggle/working/flow_imagenet.pt')) 
    i3d.to(device)
    i3d.eval()
    return i3d

# --- I3D Feature Extractor (1024-D) ---
class I3DFeatureExtractor(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        # Use the actual layer names from the InceptionI3d model
        self.features = torch.nn.Sequential(
            model.Conv3d_1a_7x7,
            model.MaxPool3d_2a_3x3,
            model.Conv3d_2b_1x1,
            model.Conv3d_2c_3x3,
            model.MaxPool3d_3a_3x3,
            model.Mixed_3b,
            model.Mixed_3c,
            model.MaxPool3d_4a_3x3,
            model.Mixed_4b,
            model.Mixed_4c,
            model.Mixed_4d,
            model.Mixed_4e,
            model.Mixed_4f,
            model.MaxPool3d_5a_2x2,
            model.Mixed_5b,
            model.Mixed_5c
        )

    def forward(self, x):
        x = x.unsqueeze(2)  # (N, C, 1, H, W)
        x = self.features(x)
        x = torch.mean(x, dim=[2, 3, 4])  # Global average pooling
        return x

# --- Optical Flow Image Preprocessing ---
def preprocess_flow_frame(frame_path):
    # Open the optical flow image (typically has 2 channels: x and y directions)
    img = Image.open(frame_path).convert('RGB')
    
    # Extract only the x and y flow channels (typically R and G channels)
    # Skip the third channel which might be unused in flow data
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # I3D expects 224x224
        transforms.ToTensor(),
    ])
    
    tensor = transform(img)
    # Use only the first two channels (x and y flow)
    flow_tensor = tensor[:2, :, :]
    
    # Normalize to [-1, 1] range
    flow_tensor = (flow_tensor * 2) - 1
    
    return flow_tensor.to(device)

# --- Feature Extraction ---
def extract_i3d_features(model, dataset_path, output_dir):
    extractor = I3DFeatureExtractor(model)
    
    for video_folder in tqdm(sorted(os.listdir(dataset_path))):
        flow_dir = os.path.join(dataset_path, video_folder, 'optical_flow')
        if not os.path.exists(flow_dir):
            print(f"No optical flow directory for {video_folder}, skipping.")
            continue
            
        flow_paths = sorted([os.path.join(flow_dir, f) 
                           for f in os.listdir(flow_dir) 
                           if f.lower().endswith(('.jpg','.png','.jpeg'))])
        
        # Create output directory for this video's features
        video_output_dir = os.path.join(output_dir, video_folder)
        os.makedirs(video_output_dir, exist_ok=True)
        
        for flow_idx, flow_path in enumerate(flow_paths):
            try:
                flow = preprocess_flow_frame(flow_path)
                with torch.no_grad():
                    features = extractor(flow.unsqueeze(0)).cpu().numpy()  # (1,1024)
                
                # Save each frame's features as separate numpy file
                flow_features = features.squeeze()
                np.save(os.path.join(video_output_dir, f"flow_{flow_idx:04d}_i3d.npy"), flow_features)
                
            except Exception as e:
                print(f"Error processing {flow_path}: {e}")
                continue

if __name__ == "__main__":
    print("Loading Flow I3D model...")
    i3d = load_i3d_model()
    
    print("Extracting 1024-D optical flow features...")
    extract_i3d_features(i3d, dataset_path, output_dir)
    
    # Verify feature dimensions
    sample_video = os.listdir(output_dir)[0]
    sample_frame = os.listdir(os.path.join(output_dir, sample_video))[0]
    sample_features = np.load(os.path.join(output_dir, sample_video, sample_frame))
    print(f"\nFeature verification:")
    print(f"Shape: {sample_features.shape}")  # Should be (1024,)
    print(f"First 5 dimensions: {sample_features[:5]}")
    
    print(f"\n✅ 1024-D optical flow features saved to individual files in {output_dir}")

Using device: cuda
Loading Flow I3D model...


<ipython-input-5-a02a88441b98>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  i3d.load_state_dict(torch.load('/kaggle/working/flow_imagenet.pt'))


Extracting 1024-D optical flow features...


100%|██████████| 330/330 [1:52:51<00:00, 20.52s/it]


Feature verification:
Shape: (1024,)
First 5 dimensions: [0.21002516 0.         0.24525137 0.         0.0009796 ]

✅ 1024-D optical flow features saved to individual files in /kaggle/working/I3D_Features_flowframetrain


# CONCADNATION

In [6]:
import os
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as transforms
from pytorch_i3d import InceptionI3d

# --- Config ---
dataset_path = '/kaggle/input/shanghaitech-anomaly-detection/dataset/mp'
output_dir = '/kaggle/working/I3D_Features_combined'
os.makedirs(output_dir, exist_ok=True)

# --- Device Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- I3D Models Loader ---
def load_i3d_models():
    # Load RGB I3D
    i3d_rgb = InceptionI3d(400, in_channels=3)
    i3d_rgb.load_state_dict(torch.load('/kaggle/working/rgb_imagenet.pt'))
    i3d_rgb.to(device)
    i3d_rgb.eval()
    
    # Load Flow I3D
    i3d_flow = InceptionI3d(400, in_channels=2)
    i3d_flow.load_state_dict(torch.load('/kaggle/working/flow_imagenet.pt'))
    i3d_flow.to(device)
    i3d_flow.eval()
    
    return i3d_rgb, i3d_flow

# --- I3D Feature Extractor (1024-D) ---
class I3DFeatureExtractor(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.features = torch.nn.Sequential(
            model.Conv3d_1a_7x7,
            model.MaxPool3d_2a_3x3,
            model.Conv3d_2b_1x1,
            model.Conv3d_2c_3x3,
            model.MaxPool3d_3a_3x3,
            model.Mixed_3b,
            model.Mixed_3c,
            model.MaxPool3d_4a_3x3,
            model.Mixed_4b,
            model.Mixed_4c,
            model.Mixed_4d,
            model.Mixed_4e,
            model.Mixed_4f,
            model.MaxPool3d_5a_2x2,
            model.Mixed_5b,
            model.Mixed_5c
        )
        
        # Add hooks to capture intermediate outputs
        self.activation = {}
        for name, layer in self.features.named_children():
            layer.register_forward_hook(self._get_activation(name))
    
    def _get_activation(self, name):
        def hook(module, input, output):
            self.activation[name] = output
        return hook

    def forward(self, x):
        x = x.unsqueeze(2)  # (N, C, 1, H, W)
        
        # Print input shape
        print(f"Input shape: {x.shape}")
        
        # Pass through network
        x = self.features(x)
        
        # Print shape before pooling
        print(f"Feature shape before pooling: {x.shape}")
        
        # Global average pooling
        x = torch.mean(x, dim=[2, 3, 4])
        
        # Print shape after pooling
        print(f"Feature shape after pooling: {x.shape}")
        
        return x

# --- Preprocessing Functions ---
def preprocess_rgb_frame(frame_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    img = Image.open(frame_path).convert('RGB')
    return transform(img).to(device)

def preprocess_flow_frame(frame_path):
    img = Image.open(frame_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    tensor = transform(img)
    flow_tensor = tensor[:2, :, :]  # Use only x and y channels
    flow_tensor = (flow_tensor * 2) - 1  # Normalize to [-1, 1]
    return flow_tensor.to(device)

# --- Feature Extraction ---
def extract_combined_features(rgb_model, flow_model, dataset_path, output_dir):
    rgb_extractor = I3DFeatureExtractor(rgb_model)
    flow_extractor = I3DFeatureExtractor(flow_model)
    
    for video_folder in tqdm(sorted(os.listdir(dataset_path))):
        # Define directories
        frame_dir = os.path.join(dataset_path, video_folder, 'frames')
        flow_dir = os.path.join(dataset_path, video_folder, 'optical_flow')
        
        if not os.path.exists(frame_dir) or not os.path.exists(flow_dir):
            print(f"Missing data directories for {video_folder}, skipping.")
            continue
            
        # Get frame paths
        frame_paths = sorted([os.path.join(frame_dir, f) 
                           for f in os.listdir(frame_dir) 
                           if f.lower().endswith(('.jpg','.png','.jpeg'))])
        
        flow_paths = sorted([os.path.join(flow_dir, f) 
                           for f in os.listdir(flow_dir) 
                           if f.lower().endswith(('.jpg','.png','.jpeg'))])
        
        # Create output directory for this video's features
        video_output_dir = os.path.join(output_dir, video_folder)
        os.makedirs(video_output_dir, exist_ok=True)
        
        # Process frames (assuming equal number of RGB and flow frames)
        for idx in range(min(len(frame_paths), len(flow_paths))):
            try:
                rgb_path = frame_paths[idx]
                flow_path = flow_paths[idx]
                
                # Print processing status
                print(f"\nProcessing frame {idx} from {video_folder}")
                
                # Process RGB frame
                print("RGB Feature Extraction:")
                rgb_frame = preprocess_rgb_frame(rgb_path)
                print(f"RGB tensor shape: {rgb_frame.shape}")
                
                with torch.no_grad():
                    rgb_features = rgb_extractor(rgb_frame.unsqueeze(0))
                    print(f"RGB features shape (PyTorch): {rgb_features.shape}")
                    rgb_features_np = rgb_features.cpu().numpy()
                    print(f"RGB features shape (NumPy): {rgb_features_np.shape}")
                
                # Process Flow frame
                print("\nOptical Flow Feature Extraction:")
                flow_frame = preprocess_flow_frame(flow_path)
                print(f"Flow tensor shape: {flow_frame.shape}")
                
                with torch.no_grad():
                    flow_features = flow_extractor(flow_frame.unsqueeze(0))
                    print(f"Flow features shape (PyTorch): {flow_features.shape}")
                    flow_features_np = flow_features.cpu().numpy()
                    print(f"Flow features shape (NumPy): {flow_features_np.shape}")
                
                # Concatenate features
                combined_features = np.concatenate((rgb_features_np.squeeze(), flow_features_np.squeeze()), axis=0)
                print(f"\nCombined features shape: {combined_features.shape}")  # Should be (2048,)
                
                # Save combined features
                np.save(os.path.join(video_output_dir, f"combined_{idx:04d}_i3d.npy"), combined_features)
                
                # Only print detailed shape info for the first frame
                if idx > 0:
                    break
                
            except Exception as e:
                print(f"Error processing frame {idx} from {video_folder}: {e}")
                continue

    # Print intermediate feature maps for a single sample
    print("\n--- Detailed Feature Map Shapes ---")
    for name, activation in rgb_extractor.activation.items():
        print(f"RGB Layer {name}: {activation.shape}")
    
    for name, activation in flow_extractor.activation.items():
        print(f"Flow Layer {name}: {activation.shape}")

if __name__ == "__main__":
    print("Loading I3D models...")
    i3d_rgb, i3d_flow = load_i3d_models()
    
    print("Extracting and combining 1024-D RGB and flow features...")
    extract_combined_features(i3d_rgb, i3d_flow, dataset_path, output_dir)
    
    # Verify final output
    try:
        sample_video = os.listdir(output_dir)[0]
        sample_frame = os.listdir(os.path.join(output_dir, sample_video))[0]
        sample_features = np.load(os.path.join(output_dir, sample_video, sample_frame))
        print(f"\nFinal Feature Verification:")
        print(f"Shape: {sample_features.shape}")  # Should be (2048,)
        print(f"First 5 RGB dimensions: {sample_features[:5]}")
        print(f"First 5 flow dimensions: {sample_features[1024:1029]}")
        
        print(f"\n✅ 2048-D combined features saved to {output_dir}")
    except Exception as e:
        print(f"Error during verification: {e}")

Using device: cuda
Loading I3D models...


<ipython-input-6-8c33e72fda6f>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  i3d_rgb.load_state_dict(torch.load('/kaggle/working/rgb_imagenet.pt'))
<ipython-input-6-8c33

Extracting and combining 1024-D RGB and flow features...


  0%|          | 1/330 [00:00<00:36,  8.91it/s]


Processing frame 0 from 01_001
RGB Feature Extraction:
RGB tensor shape: torch.Size([3, 224, 224])
Input shape: torch.Size([1, 3, 1, 224, 224])
Feature shape before pooling: torch.Size([1, 1024, 1, 7, 7])
Feature shape after pooling: torch.Size([1, 1024])
RGB features shape (PyTorch): torch.Size([1, 1024])
RGB features shape (NumPy): (1, 1024)

Optical Flow Feature Extraction:
Flow tensor shape: torch.Size([2, 224, 224])
Input shape: torch.Size([1, 2, 1, 224, 224])
Feature shape before pooling: torch.Size([1, 1024, 1, 7, 7])
Feature shape after pooling: torch.Size([1, 1024])
Flow features shape (PyTorch): torch.Size([1, 1024])
Flow features shape (NumPy): (1, 1024)

Combined features shape: (2048,)

Processing frame 1 from 01_001
RGB Feature Extraction:
RGB tensor shape: torch.Size([3, 224, 224])
Input shape: torch.Size([1, 3, 1, 224, 224])
Feature shape before pooling: torch.Size([1, 1024, 1, 7, 7])
Feature shape after pooling: torch.Size([1, 1024])
RGB features shape (PyTorch): torc

...

Feature shape before pooling: torch.Size([1, 1024, 1, 7, 7])
Feature shape after pooling: torch.Size([1, 1024])
RGB features shape (PyTorch): torch.Size([1, 1024])
RGB features shape (NumPy): (1, 1024)

Optical Flow Feature Extraction:
Flow tensor shape: torch.Size([2, 224, 224])
Input shape: torch.Size([1, 2, 1, 224, 224])
Feature shape before pooling: torch.Size([1, 1024, 1, 7, 7])
Feature shape after pooling: torch.Size([1, 1024])
Flow features shape (PyTorch): torch.Size([1, 1024])
Flow features shape (NumPy): (1, 1024)

Combined features shape: (2048,)

Processing frame 0 from 13_002
RGB Feature Extraction:
RGB tensor shape: torch.Size([3, 224, 224])
Input shape: torch.Size([1, 3, 1, 224, 224])
Feature shape before pooling: torch.Size([1, 1024, 1, 7, 7])
Feature shape after pooling: torch.Size([1, 1024])
RGB features shape (PyTorch): torch.Size([1, 1024])
RGB features shape (NumPy): (1, 1024)

Optical Flow Feature Extraction:
Flow tensor shape: torch.Size([2, 224, 224])
Input shap

 99%|█████████▉| 328/330 [00:35<00:00, 10.79it/s]

RGB tensor shape: torch.Size([3, 224, 224])
Input shape: torch.Size([1, 3, 1, 224, 224])
Feature shape before pooling: torch.Size([1, 1024, 1, 7, 7])
Feature shape after pooling: torch.Size([1, 1024])
RGB features shape (PyTorch): torch.Size([1, 1024])
RGB features shape (NumPy): (1, 1024)

Optical Flow Feature Extraction:
Flow tensor shape: torch.Size([2, 224, 224])
Input shape: torch.Size([1, 2, 1, 224, 224])
Feature shape before pooling: torch.Size([1, 1024, 1, 7, 7])
Feature shape after pooling: torch.Size([1, 1024])
Flow features shape (PyTorch): torch.Size([1, 1024])
Flow features shape (NumPy): (1, 1024)

Combined features shape: (2048,)

Processing frame 1 from 13_004
RGB Feature Extraction:
RGB tensor shape: torch.Size([3, 224, 224])
Input shape: torch.Size([1, 3, 1, 224, 224])
Feature shape before pooling: torch.Size([1, 1024, 1, 7, 7])
Feature shape after pooling: torch.Size([1, 1024])
RGB features shape (PyTorch): torch.Size([1, 1024])
RGB features shape (NumPy): (1, 1024)


100%|██████████| 330/330 [00:35<00:00,  9.33it/s]

Feature shape before pooling: torch.Size([1, 1024, 1, 7, 7])
Feature shape after pooling: torch.Size([1, 1024])
Flow features shape (PyTorch): torch.Size([1, 1024])
Flow features shape (NumPy): (1, 1024)

Combined features shape: (2048,)

Processing frame 1 from 13_006
RGB Feature Extraction:
RGB tensor shape: torch.Size([3, 224, 224])
Input shape: torch.Size([1, 3, 1, 224, 224])
Feature shape before pooling: torch.Size([1, 1024, 1, 7, 7])
Feature shape after pooling: torch.Size([1, 1024])
RGB features shape (PyTorch): torch.Size([1, 1024])
RGB features shape (NumPy): (1, 1024)

Optical Flow Feature Extraction:
Flow tensor shape: torch.Size([2, 224, 224])
Input shape: torch.Size([1, 2, 1, 224, 224])
Feature shape before pooling: torch.Size([1, 1024, 1, 7, 7])
Feature shape after pooling: torch.Size([1, 1024])
Flow features shape (PyTorch): torch.Size([1, 1024])
Flow features shape (NumPy): (1, 1024)

Combined features shape: (2048,)

Processing frame 0 from 13_007
RGB Feature Extractio